In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
my_face = pd.read_csv('../database/analytical_take_home_data_v3.csv')

In [13]:
my_face.head()

,treatment,company_size,campaign_spend,campaign_budget
0,False,small,10.4477,3.9035
1,False,medium,3.7776,1.9872
2,False,medium,46.1880,55.4523
3,False,small,6.6271,6.5136
4,False,small,92.3405,83.1018


1. How many campaigns have overspend of greater than 1% of their budget in the
control group? In the treatment group?

First, I will create a dataframe called **control** for the control group, and another dataframe called **treatment** for the new product.

In [49]:
 control = my_face.query("treatment == False")

In [31]:
treatment = my_face.query("treatment == True").reset_index()

I'm going to add a column, called **overspending**, with booleans as entries, where True indicates if a company has and overspend greater than 1% of its budget, and False if there is not such overspend, this in order to count how many companies present overspending.

In [20]:
def overspending(data):
    return sum(1 for i in range(data.shape[0]) 
        if data.campaign_spend[i] > data.campaign_budget[i]*1.01)

In [45]:
def overspend_tag(data):
    return [True if data.campaign_spend[i] > data.campaign_budget[i]*1.01 else False 
            for i in range(data.shape[0])]

In [55]:
control['overspending'] = overspend_tag(control)

/tmp/ipykernel_1049044/1141644292.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['overspending'] = overspend_tag(control)


In [74]:
treatment['overspending'] = overspend_tag(treatment)

In [86]:
control.query('overspending == True').shape[0]

5716

In [87]:
treatment.query('overspending == True').shape[0]

5180

As we can see, with the old producto there are 5716 companies withe overspending, meanwhile with the new product there are 5180 companies, which make a difference of 536 companies.

2. Was the new product effective at reducing overspend, and was it more or less
effective depending on the company size? Put together an analysis describing
how the treatment affected overspend.

With the treatment, 536 companies don't present overspending, now it's necessary look the differences between control and treatment groups according to the size of the companies.

In [71]:
def overspending_by_size(data):
    sizes = ['small', 'medium', 'large'] 
    return {s:
            control[np.logical_and(data.company_size == s,
                                   data.overspending == True)].shape[0] 
            for s in sizes}

In [89]:
control_by_size = overspending_by_size(control)

In [90]:
treatment_by_size = overspending_by_size(treatment)

/tmp/ipykernel_1049044/271622678.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  control[np.logical_and(data.company_size == s,
